In [1]:
import os
import geopandas as gpd
import pandas as pd
from scipy.integrate import quad_vec
from scipy.stats import uniform
import scipy.stats as stats
import numpy as np
import faker
from faker.providers.person.en import Provider
import datetime
import matplotlib.pyplot as plt
import random
import camelot
import json
#import pytess
from shapely import geometry
from shapely.geometry import Polygon, Point

data_path = "../../data/"

### Creation of extraction configuration for each page
Most pages can be parsed by using the cgfn format, but there are still a lot of
pages with small differences (length of table, height of table headers, etc.)

In [5]:
###### Different configurations when extracting the tables
cgfn = [['0,510,700,80'],['127,310,345,380,410,450']]

cgf1 = [['0,410,700,80'],['127,310,345,380,410,450']] #h1 and h3...
cgf2 = [['0,450,700,80'],['127,310,345,380,410,450']] #h3 with date...investigator..
cgf3 = [['0,510,700,80'],['127,310,345,380,410,455']]
cgf4 = [['0,510,700,80'],['127,300,315,340,380,440']]
cgf5 = [['0,510,700,80'],['127,310,345,380,420,450']]
cgf6 = [['0,410,700,80'],['127,330,350,390,420,460']] #h1 and h3... [112-)
cgf7 = [['0,450,700,80'],['127,330,350,390,420,460']] #h3 with date...investigator..
cgf8 = [['0,510,700,80'],['127,330,350,390,420,460']]
cgf9 = [['0,520,700,80'],['127,310,345,380,410,450']]

cgf0 = [['0,450,700,80'],['150,380,410,430,480,520']]
#List for creating the dictionary of pg:configuration.
#The lists postions correspond to the configuration (pos1 = cgf1...) 
pcgf = [
    [6,110,128,139,323,469,527],
    [9,12,30,39,46,43,45,49,50,56,100,109,140,145,156,163,166,170,183,209,211
    ,222,225,227,235,239,250,259,264,274,277,285,300,303,309,313,324,328,333,338
    ,346,351,362,371,438,443,463,482,497,507,517,521,525,529,530,532,533,541,542
    ,557,560,567,572,573,575],
    ['15-29'],
    ['35-38'],
    [54],
    [],
    [112,113,120,192],
    ['114-118','121-127','193-207'],
    ['381-437']
]
#Dictionary with the page:configuration combination. For some pages a individual
#configuration had to be created. Most of the indvidual configurations keep the 
#parser from including text in the parsing process that is not part of the table
pgd = {13:[['0,450,700,80'],['150,380,410,430,480,520']],14:[['0,410,700,80'],['127,310,345,380,410,460']]
      ,34:[['0,450,700,80'],['127,300,315,340,360,440']],119:[['0,510,700,460'],['127,330,360,390,420,460']]
      ,133:[['0,410,620,80'],['127,310,345,380,410,450']],182:[['0,510,700,220'],['127,310,345,380,410,450']]
      ,191:[['0,510,700,300'],['127,310,345,380,410,450']],208:[['0,510,700,300'],['127,330,350,390,420,460']]
      ,276:[['0,510,700,300'],['127,310,345,380,410,450']],302:[['0,510,700,200'],['127,310,345,380,410,450']]
      ,327:[['0,510,700,200'],['127,310,345,380,410,450']],332:[['0,510,700,300'],['127,310,345,380,410,450']]
      ,337:[['0,510,700,450'],['127,310,345,380,410,450']],350:[['0,510,700,350'],['127,310,345,380,410,450']]
      ,357:[['0,410,620,80'],['127,310,345,380,410,450']],361:[['0,510,700,200'],['127,310,345,380,410,450']]
      ,370:[['0,510,700,300'],['127,310,345,380,410,450']],379:[['0,510,700,400'],['127,310,345,380,410,450']]
      ,380:[['0,430,700,80'],['127,310,345,380,410,450']],442:[['0,510,700,480'],['127,310,345,380,410,450']]
      ,462:[['0,510,700,200'],['127,310,345,380,410,450']],506:[['0,510,700,200'],['127,310,345,380,410,450']]
      ,516:[['0,510,700,120'],['127,310,345,380,410,450']],520:[['0,510,700,250'],['127,310,345,380,410,450']]
      ,524:[['0,510,700,420'],['127,310,345,380,410,450']],526:[['0,510,700,250'],['127,310,345,380,410,450']]
      ,578:[['0,510,700,250'],['127,310,345,380,410,450']],52:[['0,510,700,80'],['127,310,345,380,415,450']]}
file = '../Codebook_CHRIS-baseline_distribute.pdf'

lcgfs = [cgf0,cgf1,cgf2,cgf3,cgf4,cgf5,cgf6,cgf7,cgf8,cgf9]
ind = 1
#Addition of the configurations for the pages in list pcgf
for i in pcgf:
    for j in i:
        if isinstance(j, str):
            sp = j.split('-')
            for k in range(int(sp[0]),int(sp[1])+1):
                pgd[k] = lcgfs[ind]
        else:
            pgd[j] = lcgfs[ind]
    ind = ind+1
pgd

{13: [['0,450,700,80'], ['150,380,410,430,480,520']],
 14: [['0,410,700,80'], ['127,310,345,380,410,460']],
 34: [['0,450,700,80'], ['127,300,315,340,360,440']],
 119: [['0,510,700,460'], ['127,330,360,390,420,460']],
 133: [['0,410,620,80'], ['127,310,345,380,410,450']],
 182: [['0,510,700,220'], ['127,310,345,380,410,450']],
 191: [['0,510,700,300'], ['127,310,345,380,410,450']],
 208: [['0,510,700,300'], ['127,330,350,390,420,460']],
 276: [['0,510,700,300'], ['127,310,345,380,410,450']],
 302: [['0,510,700,200'], ['127,310,345,380,410,450']],
 327: [['0,510,700,200'], ['127,310,345,380,410,450']],
 332: [['0,510,700,300'], ['127,310,345,380,410,450']],
 337: [['0,510,700,450'], ['127,310,345,380,410,450']],
 350: [['0,510,700,350'], ['127,310,345,380,410,450']],
 357: [['0,410,620,80'], ['127,310,345,380,410,450']],
 361: [['0,510,700,200'], ['127,310,345,380,410,450']],
 370: [['0,510,700,300'], ['127,310,345,380,410,450']],
 379: [['0,510,700,400'], ['127,310,345,380,410,450']],


### Extraction process of the tables
The tables defined in the file pdf are extracted. Unfortunately due to the bad format the tables are in, the previous pages:configuration combination had to be created.

In [6]:
#empty pages 8,109,284,345,356,481,542,549-554 
#357-361 (STAI-Y2 Anxiety questionaire), is left out
pg_list = '6-7,9-108,110-283,285-344,346-355,362-480,482-541,543-548,555-578'.split(',')
pl = []
for p in pg_list:
    sp = p.split('-')
    for i in range(int(sp[0]),int(sp[1])+1):
        pl.append(i)

#pl = ['52']
#cgf4 = [['0,410,700,80'],['127,310,345,380,410,460']]
tables = None
ind = None
df = pd.DataFrame()

for i in pl:
    ind = pgd.get(i)

    if ind is None:
        tables = camelot.read_pdf(file, pages=str(i), flavor='stream', flag_size=True 
                                 ,split_text=True
                                 ,strip_text='\n'
                                 ,table_areas=cgfn[0]
                                 ,columns=cgfn[1])
    else:
        tables = camelot.read_pdf(file, pages=str(i), flavor='stream', flag_size=True 
                                 ,split_text=True
                                 ,strip_text='\n'
                                 ,table_areas=ind[0]
                                 ,columns=ind[1])
    df = df.append(tables[0].df)

#For debugging below:
'''camelot.plot(tables[0], kind='text').show()
camelot.plot(tables[0], kind='contour').show()
df = tables[0].df
df'''
#Replace the values for two strange variables
df.loc[df[5].eq('NA') & df[6].eq(''), 6] = 'Missing'
df.loc[df[5].eq('NA') & df[6].eq('Missing'), 5] = '-89'
df.to_csv(data_path+'uncleaned_variables.csv', index=False)
print(len(df))
df.tail()

12884


,0,1,2,3,4,5,6
12,,activities,,,,,
13,x0dd51,Have you been taking any antibiotics in,S,C,-,-89,Missing
14,,the last 3 months?,,,,1,Yes
15,,,,,,2,No
16,x0ohver,Version of Saliva collection,X,C,-,1,"Version 1 (16/01/2017, 19/02/2018)"


### Cleaning process of the dataframe

In [7]:
#read data from file if available
if os.path.isfile(data_path+'uncleaned_variables.csv'):
    df = pd.read_csv(data_path+'uncleaned_variables.csv')

#replace some strings to make transformation easier
df = df.replace({'range:':'range=','<s>':'','</s>':'','std=-':'std=0.0',
                '- -':np.nan, 'std=\.':'std=0.0'}, regex=True)
df.loc[df['5'].eq('N'), '5'] = '1'
df.loc[df['5'].eq('Y'), '5'] = '2'
df = df.replace(['', '-'], np.nan)
#Delete all variables with no value and coding
df = df.dropna(how='all', axis=0)
df = df[~(~df['0'].isna() & df['5'].isna() & df['6'].isna())]
#consider what to do with such variables
df = df[~(df['0'] == 'x0alver')]
df.to_csv(data_path+'cleaned_variables.csv', index=False)

### Collection of variables into a proper format 
(one line per variable)

In [14]:
if os.path.isfile(data_path+'cleaned_variables.csv'):
    df = pd.read_csv(data_path+'cleaned_variables.csv')
df = df.astype(str)
var = ''
du = ''
so = ''
tp = ''
ve = ''
val = ''
co = ''
variable_list = []
description_unit_dic = {}
for i, row in df.iterrows():
    #New variable detected
    if row[0] != 'nan':
        #Save all info when var changes
        if row[0] != var:
            variable_list.append(var+'|'+du+'|'+so+'|'+tp+'|'+ve+'|'+val+'|'+co)
            description_unit_dic[var]=du
            var = ''
            du = ''
            so = ''
            tp = ''
            ve = ''
            val = ''
            co = ''
        var = var+row[0]
        if row[1] != 'nan':
            du = du+row[1]
        if row[2] != 'nan':
            so = so+row[2]
        if row[3] != 'nan':
            tp = tp+row[3]
        if row[4] != 'nan':
            ve = ve+row[4]
        if row[5] != 'nan':
            val = val+row[5]+';'
        if row[6] != 'nan':
            co = co+row[6]
            #if row[5] != 'nan':
            #    co = co+'\\'
    else:
        if row[1] != 'nan':
            du = du+' '+row[1]
        if row[2] != 'nan':
            so = so+' '+row[2]
        if row[3] != 'nan':
            tp = tp+' '+row[3]
        if row[4] != 'nan':
            ve = ve+' '+row[4]
        if row[5] != 'nan':
            val = val+row[5]+';'
        if row[5] == 'nan':
            if row[6] != 'nan':
                co = co+row[6]
        else:
            if row[6] != 'nan':
                co = co+'\\'+row[6]


variable_list = variable_list[1:]
del description_unit_dic['']
#Save compact variables list
with open(data_path+'compact-variables.txt', 'a', encoding="utf-8") as f:
    for i in variable_list:
        f.write(i+'\n')
    f.close()
print(len(description_unit_dic))
print(len(variable_list))
variable_list

2768
2768


['x0_age|Age at participation – years x0_age = x0_examd - x0_birthd|D|Q|||range=18.00, 93.60; median=45.56;mean=45.34; std=16.65',
 'x0_ager|Age at participation, rounded to whol years – years x0_ager = x0_examd - x0_birthd|e D|Q|||range=18, 94; median=46; mean=45.3;std=16.7',
 'x0_birthd|Date of birth|X|D|||range=17/01/1920, 20/09/2000',
 'x0_birthm|Month of birth – month|X|Q|||range=1, 12; median=6; mean=6.3; std=3.5',
 'x0_birthy|Year of birth – year|X|Q|||range=1920, 2000; median=1969;mean=1969.6; std=16.9',
 'x0_birthpc|Place of birth|X|C||1;2;3;4;5;6;|Vinschgau district\\South Tyrol (except Vinschgau)\\Italy (except South Tyrol)\\German-speaking countries\\Other European countries\\Non-European countries',
 'x0_examd|Date of examination|X|D|||range=24/08/2011, 21/12/2018',
 'x0_examm|Month of examination – month|X|Q|||range=1, 12; median=7; mean=6.6; std=3.4',
 'x0_examy|Year of examination – year|X|Q|||range=2011, 2018; median=2015;mean=2014.9; std=2.0',
 'x0_residp|Residence mu

In [15]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False
    

def get_random_point_in_polygon(poly):
    minx, miny, maxx, maxy = poly.bounds
    while True:
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p):
            return p

### Generate data from compact variable list

In [16]:
if os.path.isfile(data_path+'compact-variables.txt'):
    variable_list = []
    with open(data_path+"compact-variables.txt", encoding="utf-8") as fp: 
        for line in fp:
            variable_list.append(line[:-1])
        fp.close()

r = -1
md = -1
me = -1
std = -1
size = 13389
dfg = pd.DataFrame()
range_values_check={}
#Generate data for the variables
for vrb in variable_list:
    #print(vrb)
    sp = vrb.split('|')
    if sp[5] == '':
        spc = sp[6].replace(';;',';').split(';')
        r = spc[0].split('=')[1].split(', ')
        r = [r[0].replace(',',''),r[1].replace(',','')]

        if ':' not in r[0] and '/' not in r[0]:
            md = float(spc[1].split('=')[1])
            me = float(spc[2].split('=')[1])
            if spc[3] != 'std=-':
                std = float(spc[3].split('=')[1])
            
            if std > 0.0:
                dist = stats.truncnorm((float(r[0]) - me) / std, (float(r[1]) - me) / std, loc=me, scale=std)
                dfg[sp[0]] = dist.rvs(size)
            else:
                dfg[sp[0]] = np.random.uniform(float(r[0]),float(r[1])+0.1,size)
            
            if '.' in r[0] or '.' in r[1]:
                dfg[sp[0]] = dfg[sp[0]].round(2)
            else:
                dfg[sp[0]] = dfg[sp[0]].round().astype('int')
                
            range_values_check[sp[0]]=[[float(r[0]),float(r[1])],md,me,std]
        else:
            if '/' in r[0]:
                dt = []
                ds = pd.to_datetime(r[0], format='%d/%m/%Y')
                de = pd.to_datetime(r[1], format='%d/%m/%Y')
                divide_by = 24 * 60 * 60 * 10**9
                start_u = ds.value // divide_by
                end_u = de.value // divide_by
                dfg[sp[0]] = pd.to_datetime(np.random.randint(start_u, end_u, size), unit="D")
            
                range_values_check[sp[0]]=[[ds,de]]
            elif ':' in r[0]:
                t = []
                for i in range(0,size):
                    t.append(np.random.randint(0,25))
                dfg[sp[0]] = t
                
                range_values_check[sp[0]]=[[0,25]]
    else:
        if sp[5][-1] == ';':
            sp[5] = sp[5][:-1]
        if len(sp[6]) > 0 and sp[6][-1] == '\\':
            sp[6] = sp[6][:-1]
        if 'range=' in sp[6]:
            #Use only range for now
            spc = sp[6].replace(';;',';')[sp[6].find('range='):].split(';')
            r = spc[0].split('=')[1].split(', ')
            r = [r[0].replace(',',''),r[1].replace(',','')]
            
            if ':' not in r[0] and '/' not in r[0]:
                md = float(spc[1].split('=')[1])
                me = float(spc[2].split('=')[1])
                if spc[3] != 'std=-':
                    std = float(spc[3].split('=')[1])

                if std > 0.0:
                    dist = stats.truncnorm((float(r[0]) - me) / std, (float(r[1]) - me) / std, loc=me, scale=std)
                    dfg[sp[0]] = dist.rvs(size)
                else:
                    dfg[sp[0]] = np.random.uniform(float(r[0]),float(r[1])+0.1,size)
                    
                if '.' in r[0] or '.' in r[1]:
                    dfg[sp[0]] = dfg[sp[0]].round(2)
                else:
                    dfg[sp[0]] = dfg[sp[0]].round().astype('int')
                
                range_values_check[sp[0]]=[[float(r[0]),float(r[1])],md,me,std]
            else:
                if '/' in r[0]:
                    dt = []
                    ds = pd.to_datetime(r[0], format='%d/%m/%Y')
                    de = pd.to_datetime(r[1], format='%d/%m/%Y')
                    divide_by = 24 * 60 * 60 * 10**9
                    start_u = ds.value // divide_by
                    end_u = de.value // divide_by
                    dfg[sp[0]] = pd.to_datetime(np.random.randint(start_u, end_u, size), unit="D")
                    
                    range_values_check[sp[0]]=[[ds,de]]
                elif ':' in r[0]:
                    t = []
                    for i in range(0,size):
                        t.append(np.random.randint(0,25))
                    dfg[sp[0]] = t
                    
                    range_values_check[sp[0]]=[[0,25]]

        else:
            cv = sp[5].split(';')
            cgv = [i for i in range(len(cv))]
            if len(cv) > 0:
                ngo = 0
                #Count negative values which are signifying missing or special values
                for n in cv:
                    if RepresentsInt(n) and int(n) < 0:
                        ngo = ngo+1
                
                #Check if some values are negative and if so give each negative one a 5% to occure. 
                #If all are negative just uniformly generate values 
                if len(cgv) != ngo and ngo > 0:
                    #Calculate remaining % for non negative values
                    pn = (1.0-(0.05*ngo))/(len(cgv)-ngo)
                    p = []
                    
                    #Generate probability vector use in generation
                    for n in cv:
                        if RepresentsInt(n) and int(n) < 0:
                            p.append(0.05)
                        else:
                            p.append(pn)
                            
                    dfg[sp[0]] = [cv[i] for i in np.random.choice(cgv, size=size, p=p)]
                else:
                    p = [1.0/len(cgv)]*len(cgv)
                    dfg[sp[0]] = [cv[i] for i in np.random.choice(cgv, size=size, p=p)]
                
                cdic = {}
                if sp[6].find('\\') != -1:
                    for ind, i in enumerate(sp[6].split('\\')):
                        cdic[cv[ind]] = i
                    description_unit_dic.update({sp[0]:[description_unit_dic[sp[0]],cdic]})
                else:
                    #for ind, i in enumerate(cv):
                    #        cdic[cv[ind]] = ''
                    for ind, i in enumerate(cv):
                        cdic[cv[ind]] = i
                    if len(sp[6]) > 0:
                        cdic[cv[0]] = sp[6]
                        
                    description_unit_dic.update({sp[0]:[description_unit_dic[sp[0]],cdic]})
            else:
                print('Categorical variable with no value detected?')


#Change 'x0lp61d' to have numerical values as keys
nd = {} 
for ind, key in enumerate(description_unit_dic['x0lp61d'][1].keys()):
    nd[str(ind)]=key
description_unit_dic.update({'x0lp61d':[description_unit_dic['x0lp61d'][0],nd]})
dfg['x0lp61d'] = dfg['x0lp61d'].replace({y:x for x,y in nd.items()})
#Generate x0alver which had missing Value column
dfg['x0alver'] = np.random.randint(1,4,size)
description_unit_dic['x0alver']=['Version of the alcohol questionnaire'
        ,{1:'Version 1 (24/08/2011, 27/03/2012)'
        ,2:'Version 2 (28/03/2012, 16/05/2014)'
        ,3:'Version 3 (19/05/2014, )'}]
#Generate first and lastname based on x0_sex
fname = []
lname = []

fake = faker.Faker()
prov = Provider(fake)
for i in dfg['x0_sex']:
    if int(i) == 1:
        fname.append(prov.first_name_male())
        lname.append(prov.last_name_male())
    elif int(i) ==  2:
        fname.append(prov.first_name_female())
        lname.append(prov.last_name_female())
dfg['firstname'] = fname
dfg['lastname'] = lname

#Generate coordinates for each person corresponding to their municipality
geod = gpd.read_file(data_path+'/shapefiles/Municipalities_polygon.shp')
geod['ID'] = geod['ID'].astype('str')
cv_dic = {1:81.0, 2:40.0, 3:94.0, 4:79.0, 5:95.0, 6:53.0, 7:108.0,
          8:101.0, 9:69.0 ,10:93.0, 11:57.0, 12:103.0, 13:75.0}
cv_dic_sw = {val : key for key, val in cv_dic.items()}

geod = geod[geod['ID'].isin(['81.0', '40.0', '94.0', '79.0', '95.0', 
                              '53.0', '108.0', '101.0', '69.0', '93.0', 
                              '57.0', '103.0', '75.0'])]
geod.index = range(len(geod))
geod = geod.to_crs(epsg=4326)

df1 = dfg.groupby('x0_residp')['x0_residp'].count()
gm = {}
for ind, i in enumerate(geod['ID']):
    ggrpp = []
    for j in range(df1[cv_dic_sw[float(i)]]):
        ggrpp.append(get_random_point_in_polygon(geod[geod['ID']==i]['geometry'][ind]))
    gm[cv_dic_sw[float(i)]] = ggrpp

#gdf = gpd.GeoDataFrame(geometry=ggrpp)

lat = []
lon = []
for i in dfg['x0_residp']:
    coords = gm[i].pop()
    #lon.append(coords.y)
    #lat.append(coords.x)
    lon.append(coords.x)
    lat.append(coords.y)

dfg['lat'] = lat
dfg['lon'] = lon

#Save generated variables
dfg.to_csv(data_path+'generated_variables.csv', index=False)
#Save dictionary with descriptions for variables and categorical values
with open(data_path+'description_unit_dic.txt', 'w', encoding="utf-8") as f:
    json.dump(description_unit_dic, f)
    f.close()
#dfg.tail()

{'x0_age': 'Age at participation – years x0_age = x0_examd - x0_birthd', 'x0_ager': 'Age at participation, rounded to whol years – years x0_ager = x0_examd - x0_birthd', 'x0_birthd': 'Date of birth', 'x0_birthm': 'Month of birth – month', 'x0_birthy': 'Year of birth – year', 'x0_birthpc': ['Place of birth', {'1': 'Vinschgau district', '2': 'South Tyrol (except Vinschgau)', '3': 'Italy (except South Tyrol)', '4': 'German-speaking countries', '5': 'Other European countries', '6': 'Non-European countries'}], 'x0_examd': 'Date of examination', 'x0_examm': 'Month of examination – month', 'x0_examy': 'Year of examination – year', 'x0_residp': ['Residence municipality', {'1': 'GLURNS', '2': 'GRAUN I. V.', '3': 'KASTELBELL', '4': 'LAAS', '5': 'LATSCH', '6': 'MALS', '7': 'MARTELL', '8': 'PRAD A. ST.', '9': 'SCHLANDERS', '10': 'SCHLUDERNS', '11': 'SCHNALS', '12': 'STILFS', '13': 'TAUFERS I. M.'}], 'x0_sex': ['Sex', {'1': 'Male', '2': 'Female'}], 'x0ge00': ['Self-reported Non-European ancestry', 

### Optional steps
An optional step that could be done is to replace the numerical categorical values with their string repersentation.

In [ ]:
if os.path.isfile(data_path+'generated_variables.csv'):
    df = pd.read_csv(data_path+'generated_variables.csv')
df = df.astype(str)
#Assumed description already loaded
for i in description_unit_dic.items():
    if type(i[1][1]) == dict:
        for key, val in i[1][1].items():
            if val:
                df[i[0]] = df[i[0]].replace({key:val})

df.to_csv(data_path+'generated_variables_chg_cats.csv', index=False)
df.head()

In [ ]:
#Check if values are decently generated
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
pdd = dfg['x0_birthy'].describe()
#print(pdd)
s = ''
for k in range_values_check:
    s = s+k+':\n'
    pdd = dfg[k].describe() 
    rvc = range_values_check[k]
    if len(rvc) < 2:
        if not isinstance(rvc[0][0], datetime.datetime):
                s = s+str(0-int(pdd[3]))+'\n'+str(24-int(pdd[7]))+'\n'
        else:
            s = s+str(rvc[0][0])+' - '+str(pdd[4])+'\n'+str(rvc[0][1])+' - '+str(pdd[5])+'\n'
    else:
        s = s+'range-diff: '+str(rvc[0][0]-pdd[3])+' '+str(rvc[0][1]-pdd[7])+'\n'
        s = s+'mean-diff: '+str(rvc[2]-pdd[1])+'\nstd-diff: '+str(rvc[3]-pdd[2])+'\n'

print(s)

## Modify data frame for evaluation purposes
Modification of the existing df and subsequent alteration of some variable, occurrences for the purpose of testing the analyse capabilities of the application.

In [14]:
df = pd.read_csv(data_path+'generated_variables.csv', parse_dates=['x0_birthd', 'x0_examd',
                                                                 'x0cs05', 'x0mt06', 'x0mt06a'])

In [15]:
df[df['x0_residp']==6]['x0lp06b']

4        33
13       44
15       40
44       36
45       38
         ..
13350    40
13357    32
13378    38
13381    43
13387    34
Name: x0lp06b, Length: 1070, dtype: int64

In [16]:
# Two regions with higher specific weight, weight kg with > 150, weight kg, bodymass index x0an03a
# and hemoglobin values (MALS:6, Kastellbell:3)
dfq = df.copy()

dfq['x0lp06b'] = dfq.apply(lambda x: x['x0lp06b']+20 if x['x0_residp'] in [6, 3] 
                           and 40 < x['x0_ager'] < 91 else x['x0lp06b'], axis=1)
dfq['x0lp61a'] = dfq.apply(lambda x: x['x0lp61a']+0.020 if x['x0_residp'] in [6, 3] 
                           and 40 < x['x0_ager'] < 91 else x['x0lp61a'], axis=1)
dfq['x0an02'] = dfq.apply(lambda x: x['x0an02']+25 if x['x0_residp'] in [6, 3] 
                          and 40 < x['x0_ager'] < 91 else x['x0an02'], axis=1)
dfq['x0an02q'] = dfq.apply(lambda x: x['x0an02q']+25 if x['x0_residp'] in [6, 3] 
                           and 40 < x['x0_ager'] < 91 else x['x0an02q'], axis=1)
dfq['x0an03a'] = dfq.apply(lambda x: x['x0an02q']+15.0 if x['x0_residp'] in [6, 3] 
                           and 40 < x['x0_ager'] < 91 else x['x0an02q'], axis=1)
print(len(dfq.index))
dfq[dfq['x0_residp'].isin([6, 3])][['x0_residp', 'x0lp06b', 'x0lp61a', 'x0an02', 'x0an02q', 'x0an03a']]

13389


,x0_residp,x0lp06b,x0lp61a,x0an02,x0an02q,x0an03a
4,6,33,1.02,82.13,94.80,94.80
13,6,64,1.04,76.43,115.96,130.96
15,6,60,1.04,106.29,104.93,119.93
27,3,67,1.04,90.53,102.86,117.86
30,3,67,1.03,105.76,81.61,96.61
...,...,...,...,...,...,...
13358,3,51,1.05,79.12,97.11,112.11
13373,3,34,1.02,92.40,77.04,77.04
13378,6,58,1.03,106.08,115.47,130.47
13381,6,43,1.02,96.21,90.85,90.85


In [17]:
#pH level x0lp53 Taufers.. and Martell
dfq['x0lp53'] = dfq.apply(lambda x: x['x0lp53']-2.0 if x['x0_residp'] in [13, 7] and x['x0lp53'] > 2.5 
                          and 59 < x['x0_ager'] else x['x0lp53'], axis=1)
dfq[dfq['x0_residp'].isin([13, 7])][['x0lp53']]

,x0lp53
6,5.50
11,6.04
18,6.11
19,4.04
22,6.58
...,...
13343,5.62
13344,6.87
13355,4.67
13360,6.59


In [18]:
# x0lp16 Cholesterol total – mg/dL, MALS:6 and Martell:7
dfq['x0lp16'] = dfq.apply(lambda x: x['x0lp16']+200.0 if x['x0_residp'] in [6, 7] and x['x0lp16'] < 430.0 
                          and 40 < x['x0_ager'] < 91 else x['x0lp16'], axis=1)
# x0lp17 HDL Cholesterol mg/dL, MALS:6 and Martell:7
dfq['x0lp17'] = dfq.apply(lambda x: x['x0lp17']+45.0 if x['x0_residp'] in [6, 7] and x['x0lp17'] < 120 
                          and 40 < x['x0_ager'] < 91 else x['x0lp17'], axis=1)
# x0lp18 HDL Cholesterol mg/dL, MALS:6 and Martell:7
dfq['x0lp18'] = dfq.apply(lambda x: x['x0lp18']+85.0 if x['x0_residp'] in [6, 7] and x['x0lp18'] < 310 
                          and 40 < x['x0_ager'] < 91 else x['x0lp18'], axis=1)
dfq[dfq['x0_residp'].isin([6, 7])][['x0lp16', 'x0lp17' , 'x0lp18']]

,x0lp16,x0lp17,x0lp18
4,178.0,67.0,82.0
6,373.0,111.0,213.0
13,413.0,107.0,298.0
15,354.0,132.0,190.0
18,188.0,42.0,125.0
...,...,...,...
13357,372.0,102.0,121.0
13360,432.0,100.0,190.0
13378,346.0,118.0,237.0
13381,102.0,80.0,104.0


In [19]:
# x0lp23 Calcium (Ca) – mg/dL
dfq['x0lp23'] = dfq.apply(lambda x: x['x0lp23']+2.0 if x['x0_residp'] in [10, 12] and x['x0lp16'] > 7.0 else x['x0lp23'], axis=1)

In [20]:
dfq.to_csv(data_path+'generated_variables_questionaire.csv', index=False)

In [21]:
print(dfq[dfq['x0_residp'] == 6]['x0lp06b'].agg('mean'))

50.0196261682243


In [22]:
dfq.groupby('x0_residp').agg('mean')['x0lp06b']

x0_residp
1     36.814612
2     37.123340
3     50.276617
4     36.918264
5     36.737799
6     50.019626
7     36.725128
8     36.973659
9     36.731127
10    36.793561
11    36.877833
12    36.920076
13    36.958654
Name: x0lp06b, dtype: float64

### Modified dataframe for scalability test

In [2]:
df = pd.read_csv(data_path+'generated_variables.csv', parse_dates=['x0_birthd', 'x0_examd',
                                                                 'x0cs05', 'x0mt06', 'x0mt06a'])

In [3]:
df_repeated = pd.concat([df]*38, ignore_index=True)
len(df_repeated)

508782

In [4]:
df_repeated.to_parquet(data_path+'generated_variables_scalability.parquet')